# Qualité de l'air mesurée dans la station Châtelet

Source de données utilisée (fichiers CSV en open data): https://data.iledefrance.fr/explore/dataset/qualite-de-lair-mesuree-dans-la-station-chatelet/

**TODO**: l'index du *dataframe* devrait être constitué de la fusion des deux premières colonnes avec le bon *dtype* (un *datetime* Pandas).

## Récupération des données

In [ ]:
%matplotlib inline
#%matplotlib notebook

import matplotlib
matplotlib.rcParams['figure.figsize'] = (9, 9)

import pandas as pd

In [ ]:
def conv_func(s):
    s = s.replace('<', '')
    if s == 'ND':
        return np.nan
    elif s.strip() == '':
        return np.nan
    else:
        return float(s)

In [ ]:
url = "https://data.iledefrance.fr/explore/dataset/qualite-de-lair-mesuree-dans-la-station-chatelet/download/?format=csv&timezone=Europe/Berlin&use_labels_for_header=true"

dtype_dict = {'NO': np.float64,
              'NO2': np.float64,
              'PM10': np.float64,
              'CO2': np.float64,
              'TEMP': np.float64,
              'HUMI': np.float64}

converter_dict = {'NO': conv_func,
                  'NO2': conv_func,
                  'PM10': conv_func,
                  'CO2': conv_func,
                  'TEMP': conv_func,
                  'HUMI': conv_func}

df = pd.read_csv(url,
                 #encoding='iso-8859-1',
                 sep=';',
                 decimal=',',
                 #dtype=dtype_dict,
                 #na_values='ND',
                 converters=converter_dict)

df.astype(dtype_dict)

In [ ]:
df

In [ ]:
df.columns

In [ ]:
df.loc[:,['PM10', 'CO2']].plot(figsize=(16,6))

In [ ]:
df.loc[:,['NO', 'NO2']].plot(figsize=(16,6))

In [ ]:
df.loc[:,['TEMP', 'HUMI']].plot(figsize=(16,6))